# Class Design Exercise

<style>
section.present > section.present { 
    max-height: 90%; 
    overflow-y: scroll;
}
</style>

<small><a href="https://colab.research.google.com/github/brandeis-jdelfino/cosi-10a/blob/main/lectures/notebooks/12_class_design.ipynb">Link to interactive slides on Google Colab</a></small>

# Exercise

Create a program to model a library network:
* These libraries deal in books only - no need to model magazines, video, etc.
* There are multiple library branches, each with their own book inventory.
* Library patrons have accounts in the library network, and can have books out on loan.
* Data on books, branches, and patrons will be loaded from files.

Our program should be able to:
* List the branches and their info.
* Print out all the available and/or checked out books from a branch.
* Print out the books a patron has checked out.
* Provide very simple text search (substring matching) over book titles.

# File structure

We have 4 files:
* `branches.csv`
  * A comma-delimited file that contains one row per branch, with 3 string fields: `id`, `name`, and `address`.
* `books.json`
  * A JSON file with a list of dictionaries. Each dictionary represents a unique title. They have the following fields:
    * `id` (str)
    * `name` (str)
    * `description` (str)
    * `copies`
      * A list of dictionaries, each of which has 2 keys: `copy_id`, `branch_id`.
* `patrons.csv`
  * A comma-delimited file that contains one row per patron, with 2 string fields: `id`, and `name`.
* `checkouts.json`
  * A JSON file with a list of dictionaries. Each dictionary represents a checkout of a book copy by a patron. They have the following fields:
  * `patron_id` (str)
  * `copy_id` (str)

# Where do we start??

This is a big problem, we can't tackle it all at once.

Let's focus on a smaller piece: we'll need to load all the data in, and we'll want to provide some simple structures, in the forms of classes, to hold it.

Based on the files and data we have, let's start with these classes:
* `Branch` - a library branch
* `Patron` - a patron
* `BookCopy` - single copy of a book, containing all the info about the book

We'll leave checkouts aside for now.

There are many overly complex ways to model the relationships in this data. However, we don't know much yet. Let's keep it simple, then iterate. Each class will be self-contained, and will not refer directly to any other classes.

# Listing branches

This is the simplest operation - load `branches.csv` into a list, and then print each branch.

In [ ]:
import csv

class Branch:
    def __init__(self, id, name, address):
        self.id = id
        self.name = name
        self.address = address

    def __str__(self):
        return f"{self.name} ({self.id})"

def load_branches(filename):
    all_branches = []
    with open(filename, 'r') as f:
        reader = csv.reader(f, delimiter=',')
        for line in reader:
            all_branches.append(Branch(line[0], line[1], line[2]))
    print(f"Loaded {len(all_branches)} branches from {filename}")
    return all_branches

In [ ]:
branches = load_branches('../../snippets/library/branches.csv')
for b in branches:
    print(b)

# Listing books for a branch

Well, we need to load books if we're going to list any of them. Very similar to branches, right?

Wait, a book has multiple copies. How do we model this?

Let's try having `BookCopy` and `Book`, and each `BookCopy` will hold a reference to its `Book`.

In [ ]:
import json
class Book:
    def __init__(self, id, title, description):
        self.id = id
        self.title = title
        self.description = description

    def __str__(self):
        return f"{self.title} ({self.id})"

class BookCopy:
    def __init__(self, copy_id, book, branch_id):
        self.copy_id = copy_id
        self.book = book
        self.branch_id = branch_id

    def __str__(self):
        return f"{self.book.title} (Copy: {self.copy_id}, Book: {self.book.id})"

def load_books(filename):
    all_books = []
    with open(filename, 'r') as f:
        json_books = json.load(f)
        for json_book in json_books:
            book = Book(json_book['id'], json_book['title'],
                                    json_book['description'])

            for cp in json_book['copies']:
                all_books.append(BookCopy(cp['id'], book, cp['branch_id']))

    print(f"Loaded {len(all_books)} book copies from {filename}")
    return all_books

Great, now we can list all books for a branch:

In [ ]:
book_copies = load_books('../../snippets/library/books.json')
for b in book_copies:
    if b.branch_id == branches[0].id:
        print(b)

## What about checked out books?

We're supposed to be able to separately list checked out vs. available books.

We're not processing the checkouts yet, let's come back to this.

# Listing a patron's books

We still need to load patron data. This is straightforward, and similar to branches.

In [ ]:
class Patron:
    def __init__(self, id, name):
        self.id = id
        self.name = name

    def __str__(self):
        return f"{self.name} ({self.id})"

def load_patrons(filename):
    all_patrons = []
    with open(filename, 'r') as f:
        reader = csv.reader(f, delimiter=',')
        for line in reader:
            all_patrons.append(Patron(line[0], line[1]))
    print(f"Loaded {len(all_patrons)} patrons from {filename}")
    return all_patrons

In [ ]:
patrons = load_patrons('../../snippets/library/patrons.csv')
for p in patrons:
    print(p)

## We need to deal with checkouts

In order to list a patron's checked out books, we need to process `checkouts.json`.

Checkouts are a relationship between a `Patron` and a `BookCopy`. 

It's not clear where to put the data for a checkout:
* Give `Patron` a list of references to `BookCopy`s that are checked out
* Give `BookCopy` a reference to the `Patron` that has checked the copy out

It's a trap...

Both options are unwieldy, and violate best practices around class design. 

Classes should be "cohesive" - they should do "one thing".

A book or patron that tracks checkouts does more than one thing.

## Enter: LibraryNetwork

Let's introduce a container class to tie our different types of data together, and manage relationships between them.

In [ ]:
class LibraryNetwork:
    def __init__(self, branches, book_copies, patrons):
        self.branches = branches
        self.copies = book_copies
        self.patrons = patrons
        self.checkouts = {}

    def list_branches(self):
        return self.branches

    def list_books_for_branch(self, branch_id):
        return [cp for cp in self.copies if cp.branch_id == branch_id]
    
    def checkout(self, copy_id, patron_id):
        self.checkouts[copy_id] = patron_id

Now let's load checkout data into a network:

In [ ]:
def load_checkouts(filename, network):
    with open(filename, 'r') as f:
        checkouts = json.load(f)
        for checkout in checkouts:
            network.checkout(checkout['copy_id'], checkout['patron_id'])

In [ ]:
network = LibraryNetwork(branches, book_copies, patrons)
load_checkouts('../../snippets/library/checkouts.json', network)
print(len(network.checkouts))

In [ ]:
class LibraryNetwork:
    def __init__(self, branches, book_copies, patrons):
        self.branches = branches
        self.copies = book_copies
        self.patrons = patrons
        self.checkouts = {}

    def list_branches(self):
        return self.branches

    def list_books_for_branch(self, branch_id):
        return [cp for cp in self.copies if cp.branch_id == branch_id]
    
    def checkout(self, copy_id, patron_id):
        self.checkouts[copy_id] = patron_id
        
    def list_patrons_books(self, patron_id):
        books = []
        for copy_id in self.checkouts:
            if self.checkouts[copy_id] == patron_id:
                # we need to find the copy by copy_id... how?
                pass
        return books

## We need indices

We need to look up a `BookCopy` object by `copy_id`. We could iterate through each one to find it. 

Or we could build an index: a dictionary mapping from `copy_id` to `BookCopy` instance.

Let's update LibraryNetwork:

In [ ]:
class LibraryNetwork:
    def __init__(self, branches, book_copies, patrons):
        self.branches = {b.id: b for b in branches}
        self.copies = {cp.copy_id: cp for cp in book_copies}
        self.patrons = {p.id for p in patrons}
        self.checkouts = {}

    def list_branches(self):
        return list(self.branches.values())

    def list_books_for_branch(self, branch_id):
        return [cp for cp in self.copies.values() if cp.branch_id == branch_id]
    
    def checkout(self, copy_id, patron_id):
        self.checkouts[copy_id] = patron_id
        
    def list_patrons_books(self, patron_id):
        books = []
        for (copy_id, pid) in self.checkouts.items():
            if patron_id == pid:
                books.append(self.copies[copy_id])
        return books

In [ ]:
network = LibraryNetwork(branches, book_copies, patrons)
load_checkouts('../../snippets/library/checkouts.json', network)

for c in network.list_patrons_books(patrons[3].id):
    print(c)

## Back to listing available/checked out books

We have everything we need now.

In [ ]:
class LibraryNetwork:
    def __init__(self, branches, book_copies, patrons):
        self.branches = {b.id: b for b in branches}
        self.copies = {cp.copy_id: cp for cp in book_copies}
        self.patrons = {p.id for p in patrons}
        self.checkouts = {}

    def list_branches(self):
        return list(self.branches.values())

    def list_available_books(self, branch_id):
        # without a list comprehension
        avails = []
        for cp in self.copies.values():
            if cp.branch_id == branch_id and cp.copy_id not in self.checkouts:
                avails.append(cp)
        return avails

    def list_checked_out_books(self, branch_id):
        # with a list comprehension
        return [cp for cp in self.copies.values() if cp.branch_id == branch_id and cp.copy_id in self.checkouts]

    def checkout(self, copy_id, patron_id):
        self.checkouts[copy_id] = patron_id
        
    def list_patrons_books(self, patron_id):
        books = []
        for (copy_id, pid) in self.checkouts.items():
            if patron_id == pid:
                books.append(self.copies[copy_id])
        return books

## Last thing: search

In [ ]:
class LibraryNetwork:
    def __init__(self, branches, book_copies, patrons):
        self.branches = {b.id: b for b in branches}
        self.copies = {cp.copy_id: cp for cp in book_copies}
        self.patrons = {p.id for p in patrons}
        self.checkouts = {}

    def list_branches(self):
        return list(self.branches.values())

    def list_available_books(self, branch_id):
        # without a list comprehension
        avails = []
        for cp in self.copies.values():
            if cp.branch_id == branch_id and cp.copy_id not in self.checkouts:
                avails.append(cp)
        return avails

    def list_checked_out_books(self, branch_id):
        # with a list comprehension
        return [cp for cp in self.copies.values() if cp.branch_id == branch_id and cp.copy_id in self.checkouts]

    def list_patrons_books(self, patron_id):
        books = []
        for book_id in self.checkouts:
            if self.checkouts[book_id] == patron_id:
                books.append(self.copies[book_id])
        return books

    def book_search(self, value):
        matches = []
        for cp in self.copies.values():
            if value in cp.book.title:
                matches.append(cp)

        return matches

    def checkout(self, copy_id, patron_id):
        self.checkouts[copy_id] = patron_id

In [ ]:
network = LibraryNetwork(branches, book_copies, patrons)
load_checkouts('../../snippets/library/checkouts.json', network)

for m in network.book_search('beaver'):
    print(m)

Looks like we have to deal with duplicates. Let's return unique `Book`s.

In [ ]:
class LibraryNetwork:
    def __init__(self, branches, book_copies, patrons):
        self.branches = {b.id: b for b in branches}
        self.copies = {cp.copy_id: cp for cp in book_copies}
        self.patrons = {p.id for p in patrons}
        self.checkouts = {}

    def list_branches(self):
        return list(self.branches.values())

    def list_available_books(self, branch_id):
        # without a list comprehension
        avails = []
        for cp in self.copies.values():
            if cp.branch_id == branch_id and cp.copy_id not in self.checkouts:
                avails.append(cp)
        return avails

    def list_checked_out_books(self, branch_id):
        # with a list comprehension
        return [cp for cp in self.copies.values() if cp.branch_id == branch_id and cp.copy_id in self.checkouts]

    def list_patrons_books(self, patron_id):
        books = []
        for book_id in self.checkouts:
            if self.checkouts[book_id] == patron_id:
                books.append(self.copies[book_id])
        return books

    def book_search(self, value):
        matches = set()
        for cp in self.copies.values():
            if value in cp.book.title:
                matches.add(cp.book)

        return list(matches)

    def checkout(self, copy_id, patron_id):
        self.checkouts[copy_id] = patron_id

In [ ]:
network = LibraryNetwork(branches, book_copies, patrons)
load_checkouts('../../snippets/library/checkouts.json', network)

for m in network.book_search('beaver'):
    print(m)

# The whole thing

[repl.it link](https://replit.com/@cosi-10a-fall23/Library-Solution)

In [ ]:
import csv
import json

class Patron:
    def __init__(self, id, name):
        self.id = id
        self.name = name

    def __str__(self):
        return f"{self.name} ({self.id})"


def load_patrons(filename):
    all_patrons = []
    with open(filename, 'r') as f:
        reader = csv.reader(f, delimiter=',')
        for line in reader:
            all_patrons.append(Patron(line[0], line[1]))
    print(f"Loaded {len(all_patrons)} patrons from {filename}")
    return all_patrons


class Branch:
    def __init__(self, id, name, address):
        self.id = id
        self.name = name
        self.address = address

    def __str__(self):
        return f"{self.name} ({self.id})"


def load_branches(filename):
    all_branches = []
    with open(filename, 'r') as f:
        reader = csv.reader(f, delimiter=',')
        for line in reader:
            all_branches.append(Branch(*line))
    print(f"Loaded {len(all_branches)} branches from {filename}")
    return all_branches


class Book:
    def __init__(self, id, title, description):
        self.id = id
        self.title = title
        self.description = description

    def __str__(self):
        return f"{self.title} ({self.id})"


class BookCopy:
    def __init__(self, copy_id, book, branch_id):
        self.copy_id = copy_id
        self.book = book
        self.branch_id = branch_id

    def __str__(self):
        return f"{self.book.title} (Copy: {self.copy_id}, Book: {self.book.id})"


def load_books(filename):
    all_books = []
    with open(filename, 'r') as f:
        json_books = json.load(f)
        for json_book in json_books:
            book = Book(json_book['id'], json_book['title'],
                                    json_book['description'])

            for cp in json_book['copies']:
                all_books.append(BookCopy(cp['id'], book, cp['branch_id']))

    print(f"Loaded {len(all_books)} book copies from {filename}")
    return all_books


class LibraryNetwork:
    def __init__(self, branches, book_copies, patrons):
        self.branches = {b.id: b for b in branches}
        self.copies = {cp.copy_id: cp for cp in book_copies}
        self.patrons = {p.id for p in patrons}
        self.checkouts = {}

    def list_branches(self):
        return list(self.branches.values())

    def list_available_books(self, branch_id):
        # without a list comprehension
        avails = []
        for cp in self.copies.values():
            if cp.branch_id == branch_id and cp.copy_id not in self.checkouts:
                avails.append(cp)
        return avails

    def list_checked_out_books(self, branch_id):
        # with a list comprehension
        return [cp for cp in self.copies.values() if cp.branch_id == branch_id and cp.copy_id in self.checkouts]

    def list_patrons_books(self, patron_id):
        books = []
        for book_id in self.checkouts:
            if self.checkouts[book_id] == patron_id:
                books.append(self.copies[book_id])
        return books

    def book_search(self, value):
        matches = set()
        for cp in self.copies.values():
            if value in cp.book.title:
                matches.add(cp.book)

        return list(matches)

    def checkout(self, copy_id, patron_id):
        self.checkouts[copy_id] = patron_id


def load_checkouts(filename, network):
    with open(filename, 'r') as f:
        checkouts = json.load(f)
        for checkout in checkouts:
            network.checkout(checkout['copy_id'], checkout['patron_id'])

In [ ]:
branches = load_branches('../../snippets/library/branches.csv')
books = load_books('../../snippets/library/books.json')
patrons = load_patrons('../../snippets/library/patrons.csv')

network = LibraryNetwork(branches, books, patrons)
load_checkouts('../../snippets/library/checkouts.json', network)

print([str(x) for x in network.list_branches()])
print()
print([str(x) for x in network.list_available_books('e1077ff6-cbc0-43ef-a791-93dbae01409e')])
print()
print([str(x) for x in network.list_checked_out_books('e1077ff6-cbc0-43ef-a791-93dbae01409e')])
print()
print([str(x) for x in network.list_patrons_books(patrons[3].id)])
print()
print([str(x) for x in network.book_search("beaver")])

# Some reflections

# Keep it simple

We kept it relatively simple, but that's harder than it looks.

Managing complexity is the biggest challenge when writing code. 

> "YAGNI" (You Ain't Gonna Need it) - _Kent Beck_

> "Premature optimization is the root of all evil" - _Donald Knuth_

> "Debugging is twice as hard as writing the code in the first place. Therefore, if you write the code as cleverly as possible, you are, by definition, not smart enough to debug it." - _Brian W. Kernighan_

The best, most experienced programmers may build **complicated systems**, but they tend to write the **simplest code**.

## Keep classes a functions small and focused

Our classes ended up pretty focused. This is good class design.

Some principles to consider:
* Modularity - separate your code into discrete parts
* Cohesion - keep classes/functions focused on one thing
* Separation of concerns - keep unrelated parts of code apart
* Loose coupling - minimize strict dependencies between different code areas
   * If changing one area of your code results in changes everywhere, you have tight coupling

## Testing

We didn't write any tests as we went along, mostly due to time constraints. 

Write tests for individual pieces as you create them. Then you'll have confidence you haven't broken them as you make changes.

## This is hard

Modeling data relationships like this, and mapping them to classes in a clean way, is hard. 

It takes practice. 

Don't be afraid to make a mistake or a mess - that's how you'll learn.